In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# jax
from jax import numpy as jnp
# numpy
import numpy as np
import random
import matplotlib.pyplot as plt
# utils
from tqdm import tqdm_notebook as tqdm
import pickle
# custom modules
from canvas import *
from placer import *
from box import *
from net import *
from neuroPlacer import *
from VAEPlacer import *

## Pseudo Testcase Generation

In [ ]:
# make some boxes and nets
box_collection = {}
net_collection = {}

num_boxes = 10
num_nets = 2

# generate 10 boxes and 3 nets
for i in range(0, num_boxes):
    net_idx = int(i % num_nets)
    box_collection[f"box_{i}"] = Box(
        box_name=f"box_{i}",
        llx=random.randint(25, 55),
        lly=random.randint(25, 55),
        width=random.randint(1, 10),
        height=random.randint(1, 10),
        net_name=f"net_{net_idx}",
    )

    if f"net_{net_idx}" not in net_collection:
        net_collection[f"net_{net_idx}"] = Net(
            net_name=f"net_{net_idx}", source_box_name=f"box_{i}", sink_box_names=[]
        )
    else:
        net_collection[f"net_{net_idx}"].add_sink_box_name(f"box_{i}")

# make a canvas
pcb_canvas = PCBCanvas(width=100, height=100, boxes=box_collection, nets=net_collection)
print("number of boxes:", len(pcb_canvas.boxes), len(box_collection))
for box in pcb_canvas.boxes.values():
    print(box)
print("number of nets:", len(pcb_canvas.nets), len(net_collection))
for net in pcb_canvas.nets.values():
    print(net.net_name, net.source_box_name, net.box_names)

pcb_canvas.plot()

print(pcb_canvas.get_incidence_matrix())

In [ ]:
# pickle the canvas
pickle_id = '09'
# with open(f"./testcase/pcb_canvas_{pickle_id}.pkl", "wb") as f:
#     pickle.dump(pcb_canvas, f)
# load the canvas
with open(f"./testcase/pcb_canvas_{pickle_id}.pkl", "rb") as f:
    pcb_canvas = pickle.load(f)

pcb_canvas.plot()
print("wirelength:\t", pcb_canvas.get_total_wirelength_from_cxy())

# Placement Stage

In [8]:
# make a placer
pcbplacer = PCBPlacer(pcb_canvas)

In [ ]:
# everything 0
dx_list = [0.0] * len(pcb_canvas.boxes)
dy_list = [0.0] * len(pcb_canvas.boxes)
# dx_list = [-0.232823, 0.491275, 0.356292, 0.847550, -0.924190, 0.989562, 0.161006, 0.864985, 0.862639, 0.860507]
# dy_list = [0.317008, 0.374289, 0.440887, -0.104168, -0.934618, 0.223912, 0.223558, 0.476076, -0.014023, 0.021435]
pcb_canvas.set_all_boxes_motion(dx_list, dy_list)
print("are all boxes in canvas?", pcb_canvas.if_all_boxes_within_boundary())
plot_id = 'plot04'
# check if directory exists
if not os.path.exists(f"./plot/{plot_id}"):
    os.makedirs(f"./plot/{plot_id}")
for i in tqdm(range(500)):
    # print("iteration:", i)
    pcb_canvas.move_all_boxes()
    pcb_canvas.plot(savefig=True, filename=f"./plot/{plot_id}/placement_%d.png" % i)
    # print("\twirelength:\t", pcb_canvas.get_total_wirelength_from_cxy())

pcb_canvas.generate_gif_from_plots(plot_id, "./placement.gif")


In [ ]:
pcb_canvas.get_total_wirelength_from_cxy()